In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
from sklearn import linear_model
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [3]:
df=pd.read_csv('../../Datasets/Twitter/Datasets with likes/final_iitb_withlikes.csv')
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred
0,0,['KathuaRape'],2019-06-08 19:45:56,The humanities department of @iitbombay had co...,ajhay9,2.0,2,0.0,-0.6369,Negative,1.0
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0,0.0,0.7003,Positive,5.0
2,2,[],2019-06-08 19:12:05,The energy burden that goes unaudited. Our wor...,ronitabardhan,10.0,2,0.0,-0.2023,Negative,2.0
3,3,[],2019-06-08 18:59:07,Dr. Puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0,0.0,0.9300,Positive,5.0
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,The study of @iitbombay shows that every brand...,CitizenKamran,3.0,1,0.0,-0.7906,Negative,1.0
5,5,[],2019-06-08 17:53:22,Complete ban on production and usage will forc...,ravishankarkm,4.0,0,0.0,-0.5859,Negative,2.0
6,6,[],2019-06-08 17:51:51,Rising electricity bills of Mumbai slums threa...,RamitDebnath,7.0,2,0.0,0.2263,Positive,4.0
7,7,[],2019-06-08 17:30:05,"Such a shocking data , so all the things we ea...",anilindia2020,2.0,0,0.0,0.4184,Positive,4.0
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,The study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182,0.0,-0.7906,Negative,1.0
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@ArvindKejriwal is the best answer to all the ...,anishkatyain,0.0,0,0.0,0.9074,Positive,5.0


In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df=df[df.likes != 'Positive']
df=df[df.retweets != 'Positive']

/Users/Manam/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [7]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n
0,0,['KathuaRape'],2019-06-08 19:45:56,The humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.0,0.000449,0.000213
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,5.0,0.000000,0.000000
2,2,[],2019-06-08 19:12:05,The energy burden that goes unaudited. Our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.0,0.000449,0.001064
3,3,[],2019-06-08 18:59:07,Dr. Puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,5.0,0.000000,0.000000
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,The study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.0,0.000225,0.000319
5,5,[],2019-06-08 17:53:22,Complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,2.0,0.000000,0.000426
6,6,[],2019-06-08 17:51:51,Rising electricity bills of Mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,4.0,0.000449,0.000745
7,7,[],2019-06-08 17:30:05,"Such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,4.0,0.000000,0.000213
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,The study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.0,0.040881,0.080983
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@ArvindKejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,5.0,0.000000,0.000000


In [8]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [9]:
df = pd.DataFrame(df.replace([np.inf, -np.inf], np.nan))
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')


In [10]:
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score
0,0,['KathuaRape'],2019-06-08 19:45:56,The humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.0,0.000449,0.000213,0.035468
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,5.0,0.000000,0.000000,0.000000
2,2,[],2019-06-08 19:12:05,The energy burden that goes unaudited. Our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.0,0.000449,0.001064,0.069521
3,3,[],2019-06-08 18:59:07,Dr. Puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,5.0,0.000000,0.000000,0.000000
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,The study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.0,0.000225,0.000319,0.026247
5,5,[],2019-06-08 17:53:22,Complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,2.0,0.000000,0.000426,0.017027
6,6,[],2019-06-08 17:51:51,Rising electricity bills of Mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,4.0,0.000449,0.000745,0.056751
7,7,[],2019-06-08 17:30:05,"Such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,4.0,0.000000,0.000213,0.008513
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,The study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.0,0.040881,0.080983,5.692162
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@ArvindKejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,5.0,0.000000,0.000000,0.000000


In [11]:
#Engagement model using logistic regression
#Binning :
#engagement is 1 if the engagement score is greater than the median
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [12]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [13]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [14]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [15]:

df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.0,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,..."
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,5.0,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free..."
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.0,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee..."
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,5.0,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h..."
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.0,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ..."
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,2.0,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ..."
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,4.0,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th..."
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,4.0,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut..."
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.0,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ..."
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,5.0,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,..."


In [16]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.0,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb..."
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,5.0,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free...","[(iit, JJ), (bombay, NN), (problem, NN), (hear..."
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.0,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una..."
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,5.0,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h...","[(dr, NN), (puniyani, NN), (great, JJ), (fight..."
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.0,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (..."
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,2.0,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ..."
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,4.0,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS..."
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,4.0,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ..."
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.0,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (..."
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,5.0,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,...","[(arvindkejriwal, NN), (best, JJS), (answer, N..."


In [17]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.0,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb...","[humanities, department, iitbombay, protest, k..."
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,5.0,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free...","[(iit, JJ), (bombay, NN), (problem, NN), (hear...","[bombay, problem, heard, plastic, guide, solut..."
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.0,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una...","[energy, burden, work, httweets, iitbombay, wi..."
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,5.0,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h...","[(dr, NN), (puniyani, NN), (great, JJ), (fight...","[dr, puniyani, fight, truth, justice, hod, iit..."
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.0,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,..."
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,2.0,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ...","[ban, production, usage, force, pmoindia, envi..."
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,4.0,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS...","[electricity, bills, slums, energy, security, ..."
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,4.0,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ...","[data, things, drink, way, eat, ಒ, ದ, ತ, ತ, ಉ,..."
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.0,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,..."
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,5.0,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,...","[(arvindkejriwal, NN), (best, JJS), (answer, N...","[arvindkejriwal, answer, questions, iitdaa, ii..."


In [18]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,humanities
1,department
2,iitbombay
3,protest
4,kathuarape
5,hindutva
6,sans
7,evidence
8,wonder
9,liberals


In [19]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,humanities,humanities,101
1,department,department,472
2,iitbombay,iitbombay,12379
3,protest,protest,613
4,kathuarape,kathuarape,2
5,hindutva,hindutva,34
6,sans,sans,116
7,evidence,evidence,80
8,wonder,wonder,41
9,liberals,liberals,28


In [20]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
10,bombay,bombay,17038
2,iitbombay,iitbombay,12379
32,iit,iit,12092
166,com,com,11970
165,twitter,twitter,11935
168,students,students,9099
310,pic,pic,4666
215,university,university,3032
93,iitmadras,iitmadras,2335
59,india,india,2255


In [21]:
noun_freq_df=noun_freq_df.drop(['level_0', 'index'], axis=1)

KeyError: "['level_0' 'index'] not found in axis"

In [22]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [23]:
#list of irreleveant tweet topics
l=['cabprotests','jamia','jnu','jamiamilia','caa','aligarhmuslimuniversity','jamiaprotest']

In [24]:
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))

In [25]:
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns,irrelevance
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.0,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb...","[humanities, department, iitbombay, protest, k...",0
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,5.0,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free...","[(iit, JJ), (bombay, NN), (problem, NN), (hear...","[bombay, problem, heard, plastic, guide, solut...",0
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.0,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una...","[energy, burden, work, httweets, iitbombay, wi...",0
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,5.0,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h...","[(dr, NN), (puniyani, NN), (great, JJ), (fight...","[dr, puniyani, fight, truth, justice, hod, iit...",0
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.0,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,2.0,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ...","[ban, production, usage, force, pmoindia, envi...",0
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,4.0,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS...","[electricity, bills, slums, energy, security, ...",0
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,4.0,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ...","[data, things, drink, way, eat, ಒ, ದ, ತ, ತ, ಉ,...",0
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.0,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,5.0,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,...","[(arvindkejriwal, NN), (best, JJS), (answer, N...","[arvindkejriwal, answer, questions, iitdaa, ii...",0


In [26]:
df[df.irrelevance!=0]

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns,irrelevance
1207,1207,"['jnu', 'jamia', 'DU', 'QSWR2020pic']",2019-06-19 04:30:11,qs world university rankings : iit mumbai on ...,09seemasharma,3.0,1.0,0.0,0.2023,Positive,4.0,0.000225,0.000319,0.026247,1,QS world University rankings IIT Mumbai on top...,"[qs, world, university, rankings, iit, mumbai,...","[(qs, JJ), (world, NN), (university, NN), (ran...","[world, university, rankings, delhi, kanpur, r...",2
3072,1856,[],2019-07-28 15:38:56,good to see stray animals in iit bombay like j...,sritharan54,0.0,0.0,0.0,0.6900,Positive,5.0,0.000000,0.000000,0.000000,0,Good to see stray animals in IIT Bombay like J...,"[good, see, stray, animals, iit, bombay, like,...","[(good, JJ), (see, NN), (stray, JJ), (animals,...","[see, animals, bombay, jnu, du]",1
3074,1858,[],2019-07-28 15:38:02,"so, like jnu, du, iit bombay too haunted by st...",sritharan54,0.0,0.0,0.0,-0.3561,Negative,2.0,0.000000,0.000000,0.000000,0,So like JNU DU IIT Bombay too haunted by stray...,"[like, jnu, du, iit, bombay, haunted, stray, a...","[(like, IN), (jnu, NN), (du, NN), (iit, NN), (...","[jnu, du, iit, bombay, animals]",1
4163,1025,[],2019-08-11 19:45:00,who is not? they have set up jnu as the perfec...,rohan679,1.0,0.0,0.0,0.3798,Positive,4.0,0.000000,0.000106,0.004257,0,Who is not They have set up JNU as the perfect...,"[set, jnu, perfect, example, obey, ready, face...","[(set, VBN), (jnu, NN), (perfect, JJ), (exampl...","[jnu, example, face, brunt]",1
4183,1045,[],2019-08-11 19:45:00,who is not? they have set up jnu as the perfec...,rohan679,1.0,0.0,0.0,0.3798,Positive,4.0,0.000000,0.000106,0.004257,0,Who is not They have set up JNU as the perfect...,"[set, jnu, perfect, example, obey, ready, face...","[(set, VBN), (jnu, NN), (perfect, JJ), (exampl...","[jnu, example, face, brunt]",1
4553,1415,['ACSLIBRARYSUMMIT'],2019-08-09 10:21:30,@iiscbangalore @iitbombay @jnu @iitmadras #acs...,balaniinfotech,0.0,0.0,0.0,0.0000,Neutral,3.0,0.000000,0.000000,0.000000,0,@iiscbangalore @iitbombay @JNU @iitmadras #ACS...,"[iiscbangalore, iitbombay, jnu, iitmadras, acs...","[(iiscbangalore, RB), (iitbombay, JJ), (jnu, N...","[jnu, iitmadras, collaboration, balaniinfotech...",1
5536,832,[],2019-09-01 18:43:30,hopefully they will not make iit bombay anothe...,ashoswai,306.0,131.0,0.0,0.4574,Positive,4.0,0.029425,0.032564,3.068042,1,Hopefully they will not make IIT Bombay anothe...,"[hopefully, make, iit, bombay, another, jnu, v...","[(hopefully, RB), (make, VB), (iit, JJ), (bomb...","[bombay, jnu, timesofindia]",1
5556,852,[],2019-09-01 18:43:30,hopefully they will not make iit bombay anothe...,ashoswai,306.0,131.0,0.0,0.4574,Positive,4.0,0.029425,0.032564,3.068042,1,Hopefully they will not make IIT Bombay anothe...,"[hopefully, make, iit, bombay, another, jnu, v...","[(hopefully, RB), (make, VB), (iit, JJ), (bomb...","[bombay, jnu, timesofindia]",1
6181,1477,['jnu'],2019-09-29 03:00:10,"have fun, makarandji. alas, the hss dept there...",RajeevSrinivasa,2.0,1.0,0.0,0.5574,Positive,4.0,0.000225,0.000213,0.021990,1,Have fun makarandji Alas the hss dept there is...,"[fun, makarandji, alas, hss, dept, jnu, south,...","[(fun, NN), (makarandji, NN), (alas, NNS), (hs...","[fun, makarandji, alas, jnu, vipers, iitmadras...",1
6256,1552,[],2019-09-17 22:31:58,jnu and iit bombay was under 200 score,dharmendrpatel_,0.0,0.0,0.0,0.0000,Neutral,3.0,0.000000,0.000000,0.000000,0,Jnu and IIT Bombay was under 200 score,"[jnu, iit, bombay, 200, score]","[(jnu, NN), (iit, NN), (bombay, VBD), (200, CD...","[jnu, iit, score]",1


In [27]:
def assignpred(n):
    if n>1 and n<2.9:
        return max((n-1),1)
    if n>3.1 and n<5:
        return min((n+1),5)
    else:
        return n
    
def convert(score):
    OldRange = 2  
    NewRange = 4  
    NewValue = (((score + 1) * NewRange) / OldRange) + 1
    return NewValue
df['pred'] = df['compound'].apply(lambda x: convert(x))
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,pred,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns,irrelevance
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,1.7262,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb...","[humanities, department, iitbombay, protest, k...",0
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,4.4006,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free...","[(iit, JJ), (bombay, NN), (problem, NN), (hear...","[bombay, problem, heard, plastic, guide, solut...",0
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,2.5954,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una...","[energy, burden, work, httweets, iitbombay, wi...",0
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,4.8600,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h...","[(dr, NN), (puniyani, NN), (great, JJ), (fight...","[dr, puniyani, fight, truth, justice, hod, iit...",0
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,1.4188,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,1.8282,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ...","[ban, production, usage, force, pmoindia, envi...",0
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,3.4526,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS...","[electricity, bills, slums, energy, security, ...",0
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,3.8368,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ...","[data, things, drink, way, eat, ಒ, ದ, ತ, ತ, ಉ,...",0
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,1.4188,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,4.8148,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,...","[(arvindkejriwal, NN), (best, JJS), (answer, N...","[arvindkejriwal, answer, questions, iitdaa, ii...",0


In [28]:
df['new_pred']=df['pred']


In [29]:
df_eng1=df[df.engagement==1]

In [30]:
#if engagement score is greater than the median, increase the positive and negative sentiment
df_eng1['new_pred']=df_eng1['pred'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
df_eng1

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,...,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns,irrelevance,new_pred
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,...,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb...","[humanities, department, iitbombay, protest, k...",0,1.0000
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,...,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una...","[energy, burden, work, httweets, iitbombay, wi...",0,1.5954
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,...,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0,1.0000
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,...,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ...","[ban, production, usage, force, pmoindia, envi...",0,1.0000
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,...,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS...","[electricity, bills, slums, energy, security, ...",0,4.4526
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,...,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ...","[data, things, drink, way, eat, ಒ, ದ, ತ, ತ, ಉ,...",0,4.8368
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,...,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0,1.0000
13,13,"['CollegeConnectProgram', 'winprizes', 'moodin...",2019-06-08 13:45:27,"the college connect program of mood indigo, ii...",iitb_moodi,5.0,0.0,0.0,0.5093,Positive,...,0.000000,0.000532,0.021283,1,The College Connect Program of Mood Indigo IIT...,"[college, connect, program, mood, indigo, iit,...","[(college, NN), (connect, NN), (program, NN), ...","[college, connect, program, mood, iit, bombay,...",0,5.0000
15,15,[],2019-06-08 06:48:55,"thanks to an iit bombay initiative, this entir...",AshutoshM86,2.0,1.0,0.0,0.4404,Positive,...,0.000225,0.000213,0.021990,1,Thanks To An IIT Bombay Initiative This Entire...,"[thanks, iit, bombay, initiative, entire, mp, ...","[(thanks, NNS), (iit, VBP), (bombay, JJ), (ini...","[thanks, mp, village, food, power, swarajyamag]",0,4.8808
18,18,"['javascript', 'jQuery', 'designpic']",2019-06-08 04:55:32,learning javascript design patterns\nhttps://w...,shroffpub,2.0,2.0,0.0,0.0000,Neutral,...,0.000449,0.000213,0.035468,1,Learning JavaScript Design Patterns @addyosman...,"[learning, javascript, design, patterns, addyo...","[(learning, VBG), (javascript, NN), (design, N...","[javascript, design, i

In [32]:
df[df.engagement==1]=df_eng1

In [33]:
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,...,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns,irrelevance,new_pred
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,...,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb...","[humanities, department, iitbombay, protest, k...",0,1.0000
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,...,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free...","[(iit, JJ), (bombay, NN), (problem, NN), (hear...","[bombay, problem, heard, plastic, guide, solut...",0,4.4006
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,...,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una...","[energy, burden, work, httweets, iitbombay, wi...",0,1.5954
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,...,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h...","[(dr, NN), (puniyani, NN), (great, JJ), (fight...","[dr, puniyani, fight, truth, justice, hod, iit...",0,4.8600
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,...,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0,1.0000
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,...,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ...","[ban, production, usage, force, pmoindia, envi...",0,1.0000
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,...,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS...","[electricity, bills, slums, energy, security, ...",0,4.4526
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,...,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ...","[data, things, drink, way, eat, ಒ, ದ, ತ, ತ, ಉ,...",0,4.8368
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,...,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0,1.0000
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,...,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,...","[(arvindkejriwal, NN), (best, JJS), (answer, N...","[arvindkejriwal, answer, questions, iitdaa, ii...",0,4.8148


In [34]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_pred']=3

In [35]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_pred']=3

In [36]:
df.rename(columns = {'new_pred':'sentiment_score'}, inplace = True) 
df

,index,hashtags,timestamp,text,screen_name,likes,retweets,uni_handle,compound,Sentiment,...,retweets_n,likes_n,engagement_score,engagement,clean_tweets,tweet_tokens,pos,nouns,irrelevance,sentiment_score
0,0,['KathuaRape'],2019-06-08 19:45:56,the humanities department of @iitbombay had co...,ajhay9,2.0,2.0,0.0,-0.6369,Negative,...,0.000449,0.000213,0.035468,1,The humanities department of @iitbombay had co...,"[humanities, department, iitbombay, conducted,...","[(humanities, NNS), (department, NN), (iitbomb...","[humanities, department, iitbombay, protest, k...",0,1.0000
1,1,[],2019-06-08 19:12:35,"iit bombay problem heard , which salt is good ...",satishgmatt,0.0,0.0,0.0,0.7003,Positive,...,0.000000,0.000000,0.000000,0,iit bombay problem heard which salt is good an...,"[iit, bombay, problem, heard, salt, good, free...","[(iit, JJ), (bombay, NN), (problem, NN), (hear...","[bombay, problem, heard, plastic, guide, solut...",0,4.4006
2,2,[],2019-06-08 19:12:05,the energy burden that goes unaudited. our wor...,ronitabardhan,10.0,2.0,0.0,-0.2023,Negative,...,0.000449,0.001064,0.069521,1,The energy burden that goes unaudited Our work...,"[energy, burden, goes, unaudited, work, httwee...","[(energy, NN), (burden, NN), (goes, VBZ), (una...","[energy, burden, work, httweets, iitbombay, wi...",0,1.5954
3,3,[],2019-06-08 18:59:07,dr. puniyani is great and will fight for truth...,ChackoattukadJ,0.0,0.0,0.0,0.9300,Positive,...,0.000000,0.000000,0.000000,0,Dr Puniyani is great and will fight for truth ...,"[dr, puniyani, great, fight, truth, justice, h...","[(dr, NN), (puniyani, NN), (great, JJ), (fight...","[dr, puniyani, fight, truth, justice, hod, iit...",0,4.8600
4,4,"['microplastics', 'worldoceansday']",2019-06-08 18:17:58,the study of @iitbombay shows that every brand...,CitizenKamran,3.0,1.0,0.0,-0.7906,Negative,...,0.000225,0.000319,0.026247,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0,1.0000
5,5,[],2019-06-08 17:53:22,complete ban on production and usage will forc...,ravishankarkm,4.0,0.0,0.0,-0.5859,Negative,...,0.000000,0.000426,0.017027,1,Complete ban on production and usage will forc...,"[complete, ban, production, usage, force, us, ...","[(complete, JJ), (ban, NN), (production, NN), ...","[ban, production, usage, force, pmoindia, envi...",0,1.0000
6,6,[],2019-06-08 17:51:51,rising electricity bills of mumbai slums threa...,RamitDebnath,7.0,2.0,0.0,0.2263,Positive,...,0.000449,0.000745,0.056751,1,Rising electricity bills of Mumbai slums threa...,"[rising, electricity, bills, mumbai, slums, th...","[(rising, VBG), (electricity, NN), (bills, NNS...","[electricity, bills, slums, energy, security, ...",0,4.4526
7,7,[],2019-06-08 17:30:05,"such a shocking data , so all the things we ea...",anilindia2020,2.0,0.0,0.0,0.4184,Positive,...,0.000000,0.000213,0.008513,1,Such a shocking data so all the things we eat ...,"[shocking, data, things, eat, n, drink, pollut...","[(shocking, VBG), (data, NNS), (things, NNS), ...","[data, things, drink, way, eat, ಒ, ದ, ತ, ತ, ಉ,...",0,4.8368
8,8,"['microplastics', 'worldoceansday']",2019-06-08 17:16:56,the study of @iitbombay shows that every brand...,Tej_AnanthKumar,761.0,182.0,0.0,-0.7906,Negative,...,0.040881,0.080983,5.692162,1,The study of @iitbombay shows that every brand...,"[study, iitbombay, shows, every, brand, salt, ...","[(study, NN), (iitbombay, NN), (shows, VBZ), (...","[study, iitbombay, brand, salt, today, minute,...",0,1.0000
9,9,"['FreeMetroForWomen', 'freebies']",2019-06-08 16:32:59,@arvindkejriwal is the best answer to all the ...,anishkatyain,0.0,0.0,0.0,0.9074,Positive,...,0.000000,0.000000,0.000000,0,@ArvindKejriwal is the best answer to all the ...,"[arvindkejriwal, best, answer, questions, iit,...","[(arvindkejriwal, NN), (best, JJS), (answer, N...","[arvindkejriwal, answer, questions, iitdaa, ii...",0,4.8148


In [37]:
df.to_csv('../../Datasets/Twitter/Weighted Sentiment/iitbWeightedSentiment.csv')